In [5]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/drive', force_remount=True)

unset HISTFILE; export PS1="root@ad86b8643d4b-932780da6e5e4f0087f648d59ed8810f: "
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/content# root@ad86b8643d4b-932780da6e5e4f0087f648d59ed8810f: umount -f /content/drive || umount /content/drive; pkill -9 -x drive
umount: /content/drive: no mount point specified.
umount: /content/drive: no mount point specified.
root@ad86b8643d4b-932780da6e5e4f0087f648d59ed8810f: pkill -9 -f /opt/google/drive/directoryprefetcher_binary
root@ad86b8643d4b-932780da6e5e4f0087f648d59ed8810f: ( while `sleep 0.5`; do if [[ -d "/content/drive" && "$(ls -A /content/drive)" != "" ]]; then echo "google.colab.drive MOUNTED"; break; fi; done ) &
[1] 152
root@ad86b8643d4b-932780da6e5e4f0087f648d59ed8810f: cat /tmp/tmps3t8m8bv/drive.fifo | head -1 | ( /opt/google/drive/drive --features=fuse_max_background:1000,max_read_qps:1000,max_write_qps:1000,max_operation_batch_size:15,max_parallel_push_task_instances:1

In [6]:
import os
os.chdir('./drive/MyDrive/python_project/dacon_hand')

In [7]:
# ! unzip ./new_open.zip -d ./open

In [8]:
# 데이터 보기
import pandas as pd
import numpy as np
from glob import glob

# 이미지데이터 로딩
from PIL import Image
import cv2
from tqdm import tqdm

# 파일경로 설정
import os
import shutil
import json

# Modeling
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential, load_model

# GPU 환경 설정
import os

# Others
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

In [9]:
data_path = './open'


train_path = data_path + '/train'
test_path = data_path + '/test'

hand_gesture = pd.read_csv(data_path + '/hand_gesture_pose.csv')
sample_submission = pd.read_csv(data_path + '/sample_submission.csv')

In [10]:
# Train 데이터에 있는 폴더를 glob로 불러와
# sorted method를 통해 숫자 순으로 정렬합니다.
train_folders = sorted(glob(train_path + '/*'), key = lambda x : int(x.split('/')[-1]))
test_folders  = sorted(glob(test_path + '/*'), key = lambda x : int(x.split('/')[-1]))
train_folders[:5]

['./open/train/0',
 './open/train/1',
 './open/train/2',
 './open/train/3',
 './open/train/4']

In [17]:

def get_sliced_img(file_path, json_path):
    print(file_path)
    print(int(file_path.split('/')[-1][0]))

    margin = 1.15
    keypoints_per_hand=21
    original_image = np.array(Image.open(file_path))

    config = json.load(open(json_path))
    keypoints = np.array(config["annotations"][int(file_path.split('/')[-1][0])]["data"])
        
    ## Get points.
    xmin = int(np.min(keypoints[:keypoints_per_hand, 0]))
    xmax = int(np.max(keypoints[:keypoints_per_hand, 0]))
    ymin = int(np.min(keypoints[:keypoints_per_hand, 1]))
    ymax = int(np.max(keypoints[:keypoints_per_hand, 1]))
    
    ## Get square.
    w = xmax - xmin
    h = ymax - ymin
    
    bbox_w = round(max([w, h]) * margin)
    bbox_h = round(max([w, h]) * margin)
    
    xmin = min(int(np.floor(xmin - 0.5 * (bbox_w - w))), original_image.shape[0])
    ymin = min(int(np.floor(ymin - 0.5 * (bbox_h - h))), original_image.shape[1])
    
    ## Slice.
    image = np.array(original_image)[ymin:ymin + bbox_h, xmin:xmin + bbox_w]
    
    path = './open/crop_test/%s' % (file_path.split('/')[-2])

    try: 
        os.makedirs(path) 
    except OSError: 
        if not os.path.isdir(path): 
            raise


    image = Image.fromarray(image)
    # plt.imshow(image)
    # plt.show()

    image.save(path + '/%s' % file_path.split('/')[-1])
    
    return image

In [ ]:
size_list = []
exceptions = []

for fold in tqdm(test_folders):

    image_paths = sorted(glob(fold + '/*.png'), key = lambda x : int(x.split('/')[-1].replace('.png','')))
    json_path   = glob(fold + '/*.json')[0]

    for file_path in image_paths:
        try:
            img = get_sliced_img(file_path, json_path)
            size_list.append(list(img.size))
        except:
            exceptions.append(file_path)

print(f"success: {len(size_list)}")
print(f"failed: {len(exceptions)}")

  0%|          | 0/217 [00:00<?, ?it/s]

./open/test/649/0.png
0
./open/test/649/1.png
1
./open/test/649/2.png
2
./open/test/649/3.png
3
./open/test/649/4.png
4
./open/test/649/5.png
5
./open/test/649/6.png
6


  0%|          | 1/217 [00:09<32:52,  9.13s/it]

./open/test/650/0.png
0
./open/test/650/1.png
1
./open/test/650/2.png
2
./open/test/650/3.png
3
./open/test/650/4.png
4
./open/test/650/5.png
5
./open/test/650/6.png
6
./open/test/650/7.png
7
./open/test/650/8.png
8
./open/test/650/9.png
9


  1%|          | 2/217 [00:19<36:06, 10.08s/it]

...

./open/test/662/0.png
0
./open/test/662/1.png
1
./open/test/662/2.png
2
./open/test/662/3.png
3
./open/test/662/4.png
4
./open/test/662/5.png
5
./open/test/662/6.png
6
./open/test/662/7.png
7
./open/test/662/8.png
8
./open/test/662/9.png
9


  6%|▋         | 14/217 [02:25<33:53, 10.02s/it]

./open/test/663/0.png
0
./open/test/663/1.png
1
./open/test/663/2.png
2
./open/test/663/3.png
3
./open/test/663/4.png
4
./open/test/663/5.png
5
./open/test/663/6.png
6
./open/test/663/7.png
7
./open/test/663/8.png
8
./open/test/663/9.png
9


  7%|▋         | 15/217 [02:36<34:53, 10.36s/it]

./open/test/664/0.png
0
./open/test/664/1.png
1
./open/test/664/2.png
2
./open/test/664/3.png
3


In [9]:

def get_sliced_img(file_path, json_path):
    print(file_path)
    print(int(file_path.split('/')[-1][0]))

    margin = 1.15
    keypoints_per_hand=21
    original_image = np.array(Image.open(file_path))

    config = json.load(open(json_path))
    keypoints = np.array(config["annotations"][int(file_path.split('/')[-1][0])]["data"])
        
    ## Get points.
    xmin = int(np.min(keypoints[:keypoints_per_hand, 0]))
    xmax = int(np.max(keypoints[:keypoints_per_hand, 0]))
    ymin = int(np.min(keypoints[:keypoints_per_hand, 1]))
    ymax = int(np.max(keypoints[:keypoints_per_hand, 1]))
    
    ## Get square.
    w = xmax - xmin
    h = ymax - ymin
    
    bbox_w = round(max([w, h]) * margin)
    bbox_h = round(max([w, h]) * margin)
    
    xmin = min(int(np.floor(xmin - 0.5 * (bbox_w - w))), original_image.shape[0])
    ymin = min(int(np.floor(ymin - 0.5 * (bbox_h - h))), original_image.shape[1])
    
    ## Slice.
    image = np.array(original_image)[ymin:ymin + bbox_h, xmin:xmin + bbox_w]
    
    path = './open/crop_train/%s' % (file_path.split('/')[-2])

    try: 
        os.makedirs(path) 
    except OSError: 
        if not os.path.isdir(path): 
            raise


    image = Image.fromarray(image)
    # plt.imshow(image)
    # plt.show()

    image.save(path + '/%s' % file_path.split('/')[-1])
    
    return image

In [11]:
size_list = []
exceptions = []

for fold in tqdm(train_folders):

    image_paths = sorted(glob(fold + '/*.png'), key = lambda x : int(x.split('/')[-1].replace('.png','')))
    json_path   = glob(fold + '/*.json')[0]

    for file_path in image_paths:
        try:
            img = get_sliced_img(file_path, json_path)
            size_list.append(list(img.size))
        except:
            exceptions.append(file_path)

print(f"success: {len(size_list)}")
print(f"failed: {len(exceptions)}")

  0%|          | 0/649 [00:00<?, ?it/s]

./open/train/0/0.png
0
./open/train/0/1.png
1
./open/train/0/2.png
2
./open/train/0/3.png
3
./open/train/0/4.png
4
./open/train/0/5.png
5
./open/train/0/6.png
6
./open/train/0/7.png
7
./open/train/0/8.png
8


  0%|          | 1/649 [00:02<26:59,  2.50s/it]

./open/train/0/9.png
9
./open/train/1/0.png
0
./open/train/1/1.png
1
./open/train/1/2.png
2
./open/train/1/3.png
3
./open/train/1/4.png
4
./open/train/1/5.png
5
./open/train/1/6.png
6


  0%|          | 2/649 [00:04<21:15,  1.97s/it]

...

./open/train/647/0.png
0
./open/train/647/1.png
1
./open/train/647/2.png
2
./open/train/647/3.png
3
./open/train/647/4.png
4
./open/train/647/5.png
5
./open/train/647/6.png
6
./open/train/647/7.png
7
./open/train/647/8.png
8
./open/train/647/9.png
9


100%|█████████▉| 648/649 [35:24<00:03,  3.40s/it]

./open/train/648/0.png
0
./open/train/648/1.png
1
./open/train/648/2.png
2
./open/train/648/3.png
3
./open/train/648/4.png
4
./open/train/648/5.png
5


100%|██████████| 649/649 [35:26<00:00,  3.28s/it]

success: 5880
failed: 8


In [ ]:
exceptions

In [ ]:
json_path

In [ ]:
train_folder = train_folders[0]

image_paths = sorted(glob(train_folder + '/*.png'), key = lambda x : int(x.split('/')[-1].replace('.png','')))
json_path   = glob(train_folder + '/*.json')[0]

In [ ]:
image_paths

In [ ]:
image_paths[0].split('/')[-2]

In [ ]:
js = json.load(open(json_path))

js["annotations"][int(image_paths[0].split('/')[-2])]['data']

In [ ]:
image_path = image_paths[0]
img = Image.open(image_path)
img_arr = np.array(img)
print(img_arr.shape)
plt.imshow(img_arr)
plt.axis('off')
plt.show()

In [ ]:
js = json.load(open(json_path))

print("json keys             : ", js.keys())

print("json actor info       : ",js.get('actor'))

print("json image info       : ",js.get('image'))

print("json annotations keys : ",js.get('annotations')[0].keys())

In [ ]:
# answers = []
# for train_folder in train_folders :
#     json_path = glob(train_folder + '/*.json')[0]
#     js = json.load(open(json_path))
#     cat = js.get('action')[0]  # 수정
#     cat_name = js.get('action')[1]  # 수정
#     answers.append([train_folder.replace(data_path,''),cat, cat_name])

# answers = pd.DataFrame(answers, columns = ['train_path','answer', 'answer_name'])
# answers

In [ ]:
# answers.to_csv('./answers.csv', index=False)

In [ ]:
answers = pd.read_csv('./answers.csv')
answers

In [ ]:
answers['answer_name'].value_counts()

In [ ]:
len(answers['answer_name'].unique())

In [ ]:
classes = pd.get_dummies(answers[['answer']], columns = ['answer']).to_numpy()

In [ ]:
classes.shape

In [ ]:
np.random.shuffle(train_folders) # 일반화 가능성 높이기

In [ ]:
# images  = []
# targets = []
# for train_folder in tqdm(train_folders) :
#     image_paths = sorted(glob(train_folder + '/*.png'), key = lambda x : int(x.split('/')[-1].replace('.png','')))
#     query_path  = train_folder.replace(data_path,'')
#     target = classes[int(train_folder.split('/')[-1])] 
#     for image_path in image_paths:
#         img = image.load_img(image_path, target_size=(224,224,3))
#         img = image.img_to_array(img)
#         img = img/255
#         images.append(img)
#         targets.append(target)

In [ ]:
# np.save('./img', np.array(images))
# np.save('./targets', np.array(targets))

In [ ]:
images = np.load('./img.npy')
targets = np.load('./targets.npy')

In [ ]:
targets

In [ ]:
X = images
print('Train X Shape : ', X.shape)

y = targets
print('Train y Shape : ', y.shape)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      random_state=2021, 
                                                      test_size=0.2, 
                                                      stratify = y)

print('X_train shape : ', X_train.shape)
print('X_valid shape : ', X_valid.shape)
print('y_train shape : ', y_train.shape)
print('y_valid shape : ', y_valid.shape)

In [ ]:
learning_rate = 5e-4
weight_decay = 0.0001
batch_size = 16
num_epochs = 100
image_size = 224  # We'll resize input images to this size
patch_size = 16  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 6
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [ ]:
! pip install -U tensorflow-addons

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
# data_augmentation = tf.keras.Sequential(
#     [
#         layers.Normalization(),
#         layers.Resizing(image_size, image_size),
#         # layers.RandomFlip("horizontal"),
#         # layers.RandomRotation(factor=0.02),
#         # layers.RandomZoom(
#         #     height_factor=0.2, width_factor=0.2
#         # ),
#     ],
#     name="data_augmentation",
# )
# # Compute the mean and the variance of the training data for normalization.
# data_augmentation.layers[0].adapt(X_train)

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size=patch_size, **kwargs):
        
        self.patch_size = patch_size
        super(Patches, self).__init__(**kwargs)

    def get_config(self):
        config = super().get_config().copy()
        # config.update({
        #     'patch_size' : self.patch_size
        # })

        return config

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
image = X_train[np.random.choice(range(X_train.shape[0]))] * 255
plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches=num_patches, projection_dim=projection_dim, **kwargs):

        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )
        super(PatchEncoder, self).__init__(**kwargs)

    def get_config(self):
        config = super().get_config().copy()
        # config.update({
        #     'num_patches' : self.num_patches,
        #     'projection' : self.projection,
        #     'position_embedding' : self.position_embedding
        # })

        return config


    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)

        return encoded

In [ ]:
num_classes = classes.shape[1]
print(num_classes)
input_shape = (224, 224, 3)

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    # augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes, activation='softmax')(features)
    # Create the Keras model.
    model = tf.keras.Model(inputs=inputs, outputs=logits)

    return model

In [ ]:
from keras.callbacks import Callback
from keras import backend as K
import math
class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, n_epochs, lr_max, lr_min, n_cycle, verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.n_epochs = n_epochs
        self.lr_max = lr_max
        self.lr_min = lr_min
        self.n_cycle = n_cycle
        self.lrates = list()
        self.verbose = verbose
        
    def cosine_annealing(self, epoch):
        
        epochs_per_cycle = math.floor(self.n_epochs/self.n_cycle)
        cos_inner = (math.pi * (epoch % (epochs_per_cycle))) / (epochs_per_cycle)
        
        return (self.lr_max - self.lr_min)/2 * (tf.math.cos(cos_inner) + 1) + self.lr_min
     
    def plot(self, epoch):
        lr = self.cosine_annealing(epoch)
        self.lrates.append(lr)
        
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.cosine_annealing(epoch)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose == 0:
            print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))
            
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [ ]:
max_rate = 5e-4
min_rate = 1e-4
cycle = 10

In [ ]:
cosine_scheduler = CosineAnnealingScheduler(num_epochs, max_rate, min_rate, cycle)

for i in range(1, num_epochs + 1):
    cosine_scheduler.plot(i)

plt.plot(cosine_scheduler.lrates)
plt.title('Cosine Annealing_Toy')
plt.xlabel('epochs'); plt.ylabel('learning_rate')
plt.grid()
plt.show()

In [ ]:
# ! pip install adabelief_tf 
# from adabelief_tf import AdaBeliefOptimizer
# AdaBelief = AdaBeliefOptimizer(learning_rate=5e-4, epsilon=1e-16, weight_decay=1e-4)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = create_vit_classifier()


optimizer = tfa.optimizers.AdamW(
    learning_rate=learning_rate, weight_decay=weight_decay
)

model.compile(
optimizer=optimizer,
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
metrics=[
    tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
    tf.keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
],
)


modelpath = "./model/vit_{epoch:02d}-{val_loss:.4f}.hdf5"

checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, 
                                                save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)


In [ ]:
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpointer, early_stopping_callback,cosine_scheduler]
)

In [ ]:
model_name = 'vit_38-0.0235.hdf5'
model = load_model('./model/%s' % model_name, custom_objects={'Patches':Patches, 'PatchEncoder':PatchEncoder})

In [ ]:
# from tensorflow.keras.preprocessing import image
# test_images  = []
# for test_folder in tqdm(test_folders, total = len(test_folders)) :
#     image_paths = sorted(glob(test_folder + '/*.png'), key = lambda x : int(x.split('/')[-1].replace('.png','')))
#     query_path  = test_folder.replace(data_path,'')
#     test_image = []
#     for image_path in image_paths:
#         img = image.load_img(image_path, target_size=(224,224,3))
#         img = image.img_to_array(img)
#         img = img/255
#         test_image.append(img)
#     test_images.append(test_image)

# test_images = np.array(test_images)
# print(test_images.shape)

In [ ]:
# print(np.array(test_images.shape))

In [ ]:
# np.save('./test_images', np.array(test_images))

In [ ]:
test_images = np.load('./test_images.npy',allow_pickle=True)
print(test_images.shape)

In [ ]:
np.array(test_images[5]).shape

In [ ]:
np.mean(model.predict(np.array(test_images[5])), axis = 0)

In [ ]:
predictions = []
for test_image in tqdm(test_images, total = len(test_images)) : 
    prediction = np.mean(model.predict(np.array(test_image)), axis = 0)
    predictions.append(prediction)

sample_submission.iloc[:,1:] = predictions
display(sample_submission.head())
sample_submission.to_csv('./vti_%s.csv' % model_name, index=False)